JsonOutputParser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [3]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

output_parser = JsonOutputParser(pydantic_object=Topic)

In [4]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [18]:
from langchain_core.prompts import ChatPromptTemplate

question = "팔이 3M인 사람을 어떻게 생각해?"

prompt = ChatPromptTemplate.from_messages(
[
    ("system", "당신은 친절한 AI 어시스턴트입니다 질문에 간결하게 대답하세요"),
    ("user", "format: {format_instructions}\nQuestion:\n{question}")
]
)

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

In [20]:
chain = prompt | model | output_parser

answer = chain.invoke({"question": question})

In [21]:
type(answer)

dict

In [22]:
print(answer)

{'description': '팔이 3미터인 사람은 비정상적으로 긴 팔을 가진 것으로, 이는 유전적 돌연변이나 특정한 의학적 상태로 인해 발생할 수 있습니다.', 'hashtags': '#긴팔 #유전적돌연변이'}
